## Dota 2: Win Probability Prediction

![Dota 2](https://i.imgur.com/XFr4HYE.jpg)

### Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [2]:
X = pd.read_csv('features.csv', index_col='match_id')
y = X['radiant_win'].apply(lambda x: int(x)).values

X_test = pd.read_csv('features_test.csv', index_col='match_id')

#### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier" 
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

In [3]:
drop_columns = ['radiant_win', 'duration', 'tower_status_radiant',
           'tower_status_dire', 'barracks_status_radiant','barracks_status_dire']

X.drop(drop_columns, axis=1, inplace=True)

In [4]:
X.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [5]:
X_null = X.fillna(value=0.0)
X_test_null = X_test.fillna(value=0.0)

In [6]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X_null)
X_test_scaled = scaler.transform(X_test_null)

#### 1. Оцените качество логистической регрессии (<code>sklearn.linear_model.LogisticRegression</code> с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [7]:
%%time

grid = {'C': np.power(10.0, np.arange(-4, 4))}

kfold = KFold(n_splits=5, shuffle=True, random_state=41)

clf = LogisticRegression(random_state=41)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=kfold)

gs.fit(X_scaled, y)

CPU times: user 2min 47s, sys: 46.2 s, total: 3min 33s
Wall time: 1min 12s


GridSearchCV(cv=KFold(n_splits=5, random_state=41, shuffle=True),
             estimator=LogisticRegression(random_state=41),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
             scoring='roc_auc')

In [8]:
gs.best_params_['C'], gs.best_score_

(0.01, 0.7165699719095546)

#### 2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [9]:
drop_columns = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero',
                'd2_hero', 'd3_hero','d4_hero', 'd5_hero']

X_dropped = X_null.drop(columns=drop_columns, axis = 1)
X_dropped.shape

(97230, 91)

In [10]:
scaler = StandardScaler()

X_scaled_dropped = scaler.fit_transform(X_dropped)

In [11]:
%%time

grid = {'C': np.power(10.0, np.arange(-3, 3))}

kfold = KFold(n_splits=5, shuffle=True, random_state=41)

clf = LogisticRegression(random_state=41)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=kfold)

gs.fit(X_scaled_dropped, y)

CPU times: user 2min 20s, sys: 40.7 s, total: 3min 1s
Wall time: 53.3 s


GridSearchCV(cv=KFold(n_splits=5, random_state=41, shuffle=True),
             estimator=LogisticRegression(random_state=41),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             scoring='roc_auc')

In [12]:
gs.best_params_['C'], gs.best_score_

(0.01, 0.7166154249595966)

#### 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [13]:
categorical_features = [cplumn for cplumn in X_null.columns if 'hero' in cplumn]
unique_heroes = np.unique(X_null[categorical_features])

categorical_features += ['lobby_type']
max(unique_heroes)

112

#### 4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [14]:
# N — количество различных героев в выборке
X_pick = np.zeros((X_null.shape[0], max(unique_heroes)))

for i, match_id in enumerate(X_null.index):
    for p in range(5):
        X_pick[i, X_null.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_null.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick = pd.DataFrame(X_pick)
X_pick.head()

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [15]:
scaler = StandardScaler()

X_scaled_dropped = pd.DataFrame(scaler.fit_transform(X_dropped))

In [16]:
X_new_features = pd.concat([X_scaled_dropped, X_pick], axis=1)

X_new_features.shape

(97230, 203)

In [17]:
%%time

grid = {'C': np.power(10.0, np.arange(-2, 1))}

kfold = KFold(n_splits=5, shuffle=True, random_state=41)

clf = LogisticRegression(random_state=42)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=kfold)

gs.fit(X_new_features, y)

/home/yaiestura/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/yaiestura/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

CPU times: user 3min 39s, sys: 50.8 s, total: 4min 30s
Wall time: 1min 26s


/home/yaiestura/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=KFold(n_splits=5, random_state=41, shuffle=True),
             estimator=LogisticRegression(random_state=42),
             param_grid={'C': array([0.01, 0.1 , 1.  ])}, scoring='roc_auc')

In [18]:
gs.best_params_['C'], gs.best_score_

(0.1, 0.7521192231190994)

#### 6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [19]:
# N — количество различных героев в выборке
X_pick_test = np.zeros((X_test.shape[0], max(unique_heroes)))

for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick_test[i, X_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, X_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick_test = pd.DataFrame(X_pick_test)
X_pick_test.head()

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
4,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
X_test_null.drop(drop_columns, axis=1, inplace=True)
X_scaled_dropped_test = pd.DataFrame(scaler.transform(X_test_null))

In [21]:
X_new_features_test = pd.concat([X_scaled_dropped_test, X_pick_test], axis=1)

X_new_features_test.shape

(17177, 203)

In [22]:
%%time

clf = LogisticRegression(C=0.1, random_state=42)
clf.fit(X_new_features, y)

y_pred = clf.predict_proba(X_new_features_test)[:, 1]

CPU times: user 18.8 s, sys: 4.07 s, total: 22.9 s
Wall time: 7.18 s


/home/yaiestura/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
print(min(y_pred), max(y_pred))

0.008432196986783573 0.996372052868455


**Public score:** ROC-AUC = 0.75529

In [24]:
submission = pd.DataFrame()

submission['match_id'] = X_test.index
submission['radiant_win'] = y_pred

submission.to_csv('submission_logreg.csv', index=None)

submission.head()

,match_id,radiant_win
0,6,0.823876
1,7,0.753068
2,10,0.187789
3,13,0.861986
4,16,0.240169


### Отчет:

- **Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?**

   Качество логистической регрессии над исходными признаками: **ROC-AUC 0.716**, качество же градиентного бустинга: **ROC-AUC 0.69**. Логистическая регрессия работает быстрее по сравнению с градиентным бустингом (~30-40 сек против 2.5-3 мин), т.к логистическая регрессия принадлжеит к классу линейных моделей.



- **Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?**

   В данном случае удаление категориальных признаков не повлияло на метрику ROC-AUC, так как модель не учитывает бесполезную информацию, содержащаюся в категоральных признаках, и использует лишь интерпретируемые отмасштабированные числовые признаки



- **Сколько различных идентификаторов героев существует в данной игре?**

  В игре существует **112** различных идентификаторов героев на момент сбора тестовых данных(2015-2016), судя по предоставленным данным, однако на 2020-2021 год число героев в игре было увеличено



- **Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?**

  При добавлении bag of words по героям улучшилось качество логистисеской регресии: **ROC-AUC 0.752 против 0.716**.
  Объяняется это тем что теперь категориальные переменные, закодированные one-hot encoding привносят дополнительную полезную информацию, в отличие от случая когда мы их считали в качестве численных признаков



- **Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?**

  Минимальное значение - **0.008** прогноза на тестовой выборке, и максимальное значение - **0.996**